In [10]:
pip install nltk


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [52]:

# Adjust the path to where your file is stored on your local system
file_path = '/Users/suryabeeraka/Downloads/583 Project Phase 2/training-Obama-Romney-tweet.xlsx'

# Loading the datasets from the Excel file
obama_tweets = pd.read_excel(file_path, sheet_name='Obama')
romney_tweets = pd.read_excel(file_path, sheet_name='Romney')

# Display the first few rows of each dataset to inspect
print("Obama Tweets:")
print(obama_tweets.head(), '\n')  # Print first 5 rows of Obama tweets

print("Romney Tweets:")
print(romney_tweets.head())  # Print first 5 rows of Romney tweets

Obama Tweets:
   Unnamed: 0                 date            time  \
0         NaN                  NaN             NaN   
1         NaN  2012-10-16 00:00:00  10:28:53-05:00   
2         NaN  2016-12-10 00:00:00  10:09:00-05:00   
3         NaN  2012-10-16 00:00:00  10:04:30-05:00   
4         NaN  2012-10-16 00:00:00  10:00:36-05:00   

                                     Anootated tweet Unnamed: 4  Unnamed: 5  
0    1: positive, -1: negative, 0: neutral, 2: mixed      Class  Your class  
1  Kirkpatrick, who wore a baseball cap embroider...          0         NaN  
2  Question: If <e>Romney</e> and <e>Obama</e> ha...          2         NaN  
3  #<e>obama</e> debates that Cracker Ass Cracker...          1         NaN  
4  RT @davewiner Slate: Blame <e>Obama</e> for fo...          2         NaN   

Romney Tweets:
   Unnamed: 0                 date            time  \
0         NaN                  NaN             NaN   
1         NaN  2012-10-16 00:00:00  09:38:08-05:00   
2         NaN 

In [21]:


# Downloading necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# Initializing the lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/suryabeeraka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/suryabeeraka/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/suryabeeraka/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/suryabeeraka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
# Load the dataset from both sheets
file_path = '/Users/suryabeeraka/Downloads/583 Project Phase 2/training-Obama-Romney-tweet.xlsx'
obama_tweets = pd.read_excel(file_path, sheet_name='Obama')
romney_tweets = pd.read_excel(file_path, sheet_name='Romney')

# Print out the column names to verify
print("Obama Tweets columns:", obama_tweets.columns.tolist())
print("Romney Tweets columns:", romney_tweets.columns.tolist())

Obama Tweets columns: ['Unnamed: 0', 'date', 'time', 'Anootated tweet', 'Unnamed: 4', 'Unnamed: 5']
Romney Tweets columns: ['Unnamed: 0', 'date', 'time', 'Anootated tweet', 'Unnamed: 4', 'Unnamed: 5']


In [47]:

def clean_data(sheet, class_column_name, sentiment_col):
    # Check if sentiment column exists, handle if not
    if sentiment_col not in sheet.columns:
        print(f"Column {sentiment_col} not found in the DataFrame.")
        return None  # or handle appropriately

    # Remove rows with mixed sentiments (class '2')
    sheet = sheet[sheet[sentiment_col].isin([-1, 0, 1])]
    
    # Drop the 'Your class' or 'Your class label' column, if necessary
    if class_column_name in sheet.columns:
        sheet = sheet.drop(columns=[class_column_name])

    # Assuming 'Anootated tweet' is the correct tweet column (watch for typos)
    sheet = sheet.rename(columns={'Anootated tweet': 'Tweet'})

    # Keeping only the 'Tweet' and specified sentiment columns for analysis
    sheet = sheet[['Tweet', sentiment_col]]
    sheet = sheet.rename(columns={sentiment_col: 'Class'})  # Standardizing the class column name for consistency

    return sheet
print("Cleaned Obama Tweets:")
print(obama_tweets_clean.head(), '\n')

print("Cleaned Romney Tweets:")
print(romney_tweets_clean.head())

Cleaned Obama Tweets:
                                               Tweet Class
1  Kirkpatrick, who wore a baseball cap embroider...     0
3  #<e>obama</e> debates that Cracker Ass Cracker...     1
5  @Hollivan @hereistheanswer  Youre missing the ...     0
7  I was raised as a Democrat  left the party yea...    -1
8  The <e>Obama camp</e> can't afford to lower ex...     0 

Cleaned Romney Tweets:
                                               Tweet Class  \
1  Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...    -1   
3  .@WardBrenda @shortwave8669 @allanbourdius you...    -1   
4  <e>Mitt Romney</e> still doesn't <a>believe</a...    -1   
5  <e>Romney</e>'s <a>tax plan</a> deserves a 2nd...    -1   
6  Hope <e>Romney</e> debate prepped w/ the same ...     1   

                                     Processed_Tweet Encoded_Class  
1  insidi ! mitt romney 's bain help philip morri...          None  
3  . @ wardbrenda @ shortwave8669 @ allanbourdiu ...          None  
4        mitt romne

In [48]:
def clean_data(sheet, tweet_column_name, class_column_name):
    # Example of checking multiple potential columns for the class, and using the first found
    potential_class_columns = ['Unnamed: 4', 'Unnamed: 5']
    for col in potential_class_columns:
        if col in sheet.columns:
            class_column_name = col
            break
    
    # Remove rows if the class column contains invalid data (example assumes valid classes are integers)
    if class_column_name in sheet.columns:
        sheet = sheet[pd.to_numeric(sheet[class_column_name], errors='coerce').notna()]
    
    # Rename the tweet column for consistency
    sheet = sheet.rename(columns={tweet_column_name: 'Tweet'})
    sheet = sheet[['Tweet', class_column_name]]
    sheet = sheet.rename(columns={class_column_name: 'Class'})  # Standardizing the class column name
    
    return sheet

# Cleaning data example
obama_tweets_clean = clean_data(obama_tweets, 'Anootated tweet', 'Class')
print("Obama Tweets cleaned data sample:")
print(obama_tweets_clean.head())

Obama Tweets cleaned data sample:
                                               Tweet Class
1  Kirkpatrick, who wore a baseball cap embroider...     0
2  Question: If <e>Romney</e> and <e>Obama</e> ha...     2
3  #<e>obama</e> debates that Cracker Ass Cracker...     1
4  RT @davewiner Slate: Blame <e>Obama</e> for fo...     2
5  @Hollivan @hereistheanswer  Youre missing the ...     0


In [49]:
def preprocess_tweet(text):
    # Convert to string in case there are non-string data types
    text = str(text)
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Tokenize the text
    words = word_tokenize(text)
    # Convert to lower case
    words = [word.lower() for word in words]
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

# Preprocess the tweets
obama_tweets_clean['Processed_Tweet'] = obama_tweets_clean['Tweet'].apply(preprocess_tweet)
romney_tweets_clean['Processed_Tweet'] = romney_tweets_clean['Tweet'].apply(preprocess_tweet)

# Preview the processed tweets
print("Processed Obama Tweets:")
print(obama_tweets_clean[['Processed_Tweet', 'Class']].head(), '\n')

print("Processed Romney Tweets:")
print(romney_tweets_clean[['Processed_Tweet', 'Class']].head())


Processed Obama Tweets:
                                     Processed_Tweet Class
1  kirkpatrick , wore basebal cap embroid barack ...     0
2  question : romney obama child-punch contest , ...     2
3  # obama debat cracker ass cracker tonight ... ...     1
4  rt @ davewin slate : blame obama four death li...     2
5  @ hollivan @ hereistheansw your miss point im ...     0 

Processed Romney Tweets:
                                     Processed_Tweet Class
1  insidi ! mitt romney 's bain help philip morri...    -1
3  . @ wardbrenda @ shortwave8669 @ allanbourdiu ...    -1
4        mitt romney still n't believ black presid .    -1
5  romney 's tax plan deserv 2nd look secret one ...    -1
6        hope romney debat prep w/ peopl last time .     1


In [27]:
# Assuming sentiment labels are in a different format and need to be encoded
def encode_sentiment(label):
    if label == "positive":
        return 1
    elif label == "negative":
        return -1
    elif label == "neutral":
        return 0
    # Add additional conditions if there are more labels
    else:
        return None

# Apply the encoding function to your sentiment column
obama_tweets_clean['Encoded_Class'] = obama_tweets_clean['Class'].apply(encode_sentiment)
romney_tweets_clean['Encoded_Class'] = romney_tweets_clean['Class'].apply(encode_sentiment)


In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Use TF-IDF to vectorize the tweets
tfidf = TfidfVectorizer(max_features=1000)  # Limiting to 1000 features for simplicity
X = tfidf.fit_transform(obama_tweets_clean['Processed_Tweet']).toarray()
y = obama_tweets_clean['Class'].astype(int)  # Ensure the class labels are integers

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          -1       0.54      0.59      0.57       385
           0       0.47      0.45      0.46       401
           1       0.59      0.56      0.58       321
           2       0.72      0.70      0.71       327

    accuracy                           0.57      1434
   macro avg       0.58      0.58      0.58      1434
weighted avg       0.57      0.57      0.57      1434



/Users/suryabeeraka/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [55]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Sample DataFrame loading step (assuming DataFrame is ready and named as obama_tweets_clean)
# obama_tweets_clean = pd.read_excel('path_to_your_file.xlsx')

# Step 1: Relabel the classes to start from 0
label_mapping = {-1: 0, 0: 1, 1: 2}
obama_tweets_clean['Class'] = obama_tweets_clean['Class'].map(label_mapping)

# Ensure all text data is string and not missing
obama_tweets_clean['Tweet'] = obama_tweets_clean['Tweet'].fillna('missing').astype(str)
tweets = obama_tweets_clean['Tweet'].values
labels = obama_tweets_clean['Class'].values

# Step 2: Tokenize and pad text data
tokenizer = Tokenizer(num_words=5000)  # Keeps only the 5000 most frequent words
tokenizer.fit_on_texts(tweets)
sequences = tokenizer.texts_to_sequences(tweets)
X = pad_sequences(sequences, maxlen=100)  # Ensures all sequences have the same length
y = np.array(labels)

# Step 3: Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes

# Step 5: Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 6: Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Step 7: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')


Epoch 1/5
90/90 [==============================] - 10s 101ms/step - loss: 0.9657 - accuracy: 0.3254 - val_loss: 0.8967 - val_accuracy: 0.3738
Epoch 2/5
90/90 [==============================] - 9s 105ms/step - loss: 0.8043 - accuracy: 0.4314 - val_loss: 0.8676 - val_accuracy: 0.4177
Epoch 3/5
90/90 [==============================] - 9s 104ms/step - loss: 0.6638 - accuracy: 0.5144 - val_loss: 0.9091 - val_accuracy: 0.4031
Epoch 4/5
90/90 [==============================] - 9s 104ms/step - loss: 0.5483 - accuracy: 0.5744 - val_loss: 0.9472 - val_accuracy: 0.4317
Epoch 5/5
45/45 [==============================] - 1s 17ms/step - loss: 1.1133 - accuracy: 0.4212
Accuracy: 42.12%


In [56]:
from tensorflow.keras.layers import Bidirectional

# Updated LSTM model with bidirectional layer and increased complexity
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Consider using early stopping and model checkpointing
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
checkpoint = ModelCheckpoint( 'best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop, checkpoint])

# Load the best model and evaluate it
model.load_weights('best_model.h5')
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')


Epoch 1/10
90/90 [==============================] - ETA: 0s - loss: 0.9525 - accuracy: 0.3323
Epoch 1: val_loss improved from inf to 0.87781, saving model to best_model.h5
90/90 [==============================] - 36s 378ms/step - loss: 0.9525 - accuracy: 0.3323 - val_loss: 0.8778 - val_accuracy: 0.3905
Epoch 2/10


/Users/suryabeeraka/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


90/90 [==============================] - ETA: 0s - loss: 0.7742 - accuracy: 0.4638
Epoch 2: val_loss improved from 0.87781 to 0.86917, saving model to best_model.h5
90/90 [==============================] - 48s 532ms/step - loss: 0.7742 - accuracy: 0.4638 - val_loss: 0.8692 - val_accuracy: 0.4107
Epoch 3/10
90/90 [==============================] - ETA: 0s - loss: 0.6195 - accuracy: 0.5416
Epoch 3: val_loss did not improve from 0.86917
90/90 [==============================] - 47s 527ms/step - loss: 0.6195 - accuracy: 0.5416 - val_loss: 0.8878 - val_accuracy: 0.4184
Epoch 4/10
90/90 [==============================] - ETA: 0s - loss: 0.5010 - accuracy: 0.5965
Epoch 4: val_loss did not improve from 0.86917
90/90 [==============================] - 47s 518ms/step - loss: 0.5010 - accuracy: 0.5965 - val_loss: 0.9903 - val_accuracy: 0.4052
Epoch 5/10
90/90 [==============================] - ETA: 0s - loss: 0.4055 - accuracy: 0.6389
Epoch 5: val_loss did not improve from 0.86917
90/90 [=========

In [57]:
def prepare_tweet_data(file_path, sheet_name, tweet_col_letter, class_col_letter, excluded_class, identifier):
    # Convert Excel column letter to index (zero-based)
    def excel_col_to_index(col_letter):
        return pd.io.excel._excel._letter_to_num(col_letter.lower()) - 1
    
    # Define column indices for pandas
    cols_to_use = f"{tweet_col_letter}:{class_col_letter}"
    
    # Load specific columns from the Excel sheet
    tweets_df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=cols_to_use)
    
    # Set meaningful column names
    tweets_df.columns = ["TweetContent", "Sentiment"]
    
    # Exclude tweets with the specified class
    tweets_df = tweets_df[tweets_df['Sentiment'] != excluded_class]
    
    # Replace missing tweet content with a placeholder
    tweets_df['TweetContent'] = tweets_df['TweetContent'].fillna("UNKNOWN")
    
    # Add a column for the candidate's name
    tweets_df['Candidate'] = identifier
    
    return tweets_df[['Candidate', 'TweetContent', 'Sentiment']]
data_source_path = "/Users/suryabeeraka/Downloads/CS583_Project/training-Obama-Romney-tweets.xlsx"
obama_data_sheet  = "Obama"   
romney_data_sheet = "Romney"
tweet_col_id, sentiment_col_id = "D", "E"  # Excel column letters for tweets and sentiment labels

# Load and process tweet data for analysis
data_obama = prepare_tweet_data(data_source_path, obama_data_sheet, tweet_col_id, sentiment_col_id, 2, 'Barack Obama')
data_romney = prepare_tweet_data(data_source_path, romney_data_sheet, tweet_col_id, sentiment_col_id, 2, 'Mitt Romney')

# Aggregate processed data from both political figures into a single data structure
combined_sentiment_data = pd.concat([data_obama, data_romney], axis=0, ignore_index=True)


# Display the first few entries for each candidate
print("Sample Tweets for Obama:")
print(all_tweets_data[all_tweets_data['Candidate'] == 'Obama'].head())

print("\nSample Tweets for Romney:")
print(all_tweets_data[all_tweets_data['Candidate'] == 'Romney'].head())

Sample Tweets for Obama:


NameError: name 'all_tweets_data' is not defined

In [58]:
import pandas as pd

def load_and_clean_tweet_data(file_path, sheet_name, tweet_col_letter, class_col_letter, excluded_classes):
    # Helper function to convert Excel column letter to index (zero-based)
    def excel_col_to_index(col_letter):
        return pd.io.excel._excel._letter_to_num(col_letter.lower()) - 1

    # Load specific columns from the Excel sheet
    cols_to_use = f"{tweet_col_letter}:{class_col_letter}"
    tweets_df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=cols_to_use)
    
    # Set meaningful column names
    tweets_df.columns = ["TweetContent", "Sentiment"]
    
    # Filter out tweets with the specified class and replace missing tweet content
    tweets_df = tweets_df[tweets_df['Sentiment'].isin(excluded_classes)].copy()
    tweets_df.loc[:, 'TweetContent'] = tweets_df['TweetContent'].fillna("UNKNOWN")
    
    return tweets_df

# Example usage:
data_source_path = "/Users/suryabeeraka/Downloads/CS583_Project/training-Obama-Romney-tweets.xlsx"
tweet_col_id, sentiment_col_id = "D", "E"

# Define the classes to include (exclude class 2)
included_classes = [0, 1, -1]

# Clean data for Obama
obama_data_sheet = "Obama"
obama_cleaned_tweets = load_and_clean_tweet_data(data_source_path, obama_data_sheet, tweet_col_id, sentiment_col_id, included_classes)
print("Cleaned Data for Obama:")
print(obama_cleaned_tweets.head())

# Clean data for Romney
romney_data_sheet = "Romney"
romney_cleaned_tweets = load_and_clean_tweet_data(data_source_path, romney_data_sheet, tweet_col_id, sentiment_col_id, included_classes)
print("Cleaned Data for Romney:")
print(romney_cleaned_tweets.head())


Cleaned Data for Obama:
                                        TweetContent Sentiment
1  Kirkpatrick, who wore a baseball cap embroider...         0
3  #<e>obama</e> debates that Cracker Ass Cracker...         1
5  @Hollivan @hereistheanswer  Youre missing the ...         0
7  I was raised as a Democrat  left the party yea...        -1
8  The <e>Obama camp</e> can't afford to lower ex...         0
Cleaned Data for Romney:
                                        TweetContent Sentiment
1  Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...        -1
3  .@WardBrenda @shortwave8669 @allanbourdius you...        -1
4  <e>Mitt Romney</e> still doesn't <a>believe</a...        -1
5  <e>Romney</e>'s <a>tax plan</a> deserves a 2nd...        -1
6  Hope <e>Romney</e> debate prepped w/ the same ...         1


In [ ]:

# Part-of-speech mapping for lemmatization
pos_mapping = {
    "N": 'n',  # Noun
    "V": 'v',  # Verb
    "J": 'a',  # Adjective
    "R": 'r'   # Adverb
}

def preprocess_and_process_tweet_data(tweets_df, identifier):
    # Ensure that modifications do not affect the original DataFrame outside this function
    tweets_df = tweets_df.copy()

    # Preprocessing tweets
    tokenizer = RegexpTokenizer(r"\w+|\d+:\d+[ap]m|…")
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    def clean_and_lemmatize(text):
        if pd.isna(text):
            text = ''  # Replace NaN with empty string
        else:
            text = text.lower().replace("'s", "").replace("'", "").replace("-", " ")
            text = re.sub(r'http[s]?://\S+', '', text)
        tokens = tokenizer.tokenize(text)
        pos_tags = pos_tag(tokens)
        lemmatized = [lemmatizer.lemmatize(token, pos_mapping.get(pos[0], 'n')) for token, pos in pos_tags if token not in stop_words]
        return ' '.join(lemmatized)

    # Apply the cleaning and lemmatization function to each tweet
    tweets_df['TweetContent'] = tweets_df['TweetContent'].apply(clean_and_lemmatize)

    # Add a column for the candidate's name
    tweets_df.loc[:, 'Candidate'] = identifier

    return tweets_df[['Candidate', 'TweetContent', 'Sentiment']]

# Example usage
data_source_path = "/Users/suryabeeraka/Downloads/CS583_Project/training-Obama-Romney-tweets.xlsx"
# Load and filter the data to exclude sentiment class 2
obama_tweets = pd.read_excel(data_source_path, sheet_name="Obama", usecols="D:E")
obama_tweets.columns = ['TweetContent', 'Sentiment']
obama_tweets = obama_tweets[obama_tweets['Sentiment'] != 2]  # Exclude class 2

romney_tweets = pd.read_excel(data_source_path, sheet_name="Romney", usecols="D:E")
romney_tweets.columns = ['TweetContent', 'Sentiment']
romney_tweets = romney_tweets[romney_tweets['Sentiment'] != 2]  # Exclude class 2

# Process and display formatted data for Obama
obama_processed_tweets = preprocess_and_process_tweet_data(obama_tweets, 'Barack Obama')
obama_processed_tweets['TweetContent'] = obama_processed_tweets['TweetContent'].str.replace(' ', ', ')
print("Processed Data for Obama:")
print(obama_processed_tweets.head())

# Process and display formatted data for Romney
romney_processed_tweets = preprocess_and_process_tweet_data(romney_tweets, 'Mitt Romney')
romney_processed_tweets['TweetContent'] = romney_processed_tweets['TweetContent'].str.replace(' ', ', ')
print("\nProcessed Data for Romney:")
print(romney_processed_tweets.head())


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization and padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data_obama['TweetContent'])
X = tokenizer.texts_to_sequences(data_obama['TweetContent'])
X = pad_sequences(X, maxlen=50)

# Prepare labels
y = pd.get_dummies(data_obama['Sentiment']).values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))  # Assuming 3 classes -1, 0, 1

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print("LSTM Accuracy Score:", accuracy)
